<a href="https://colab.research.google.com/github/tamnhu2913/BCG-GenAI/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Preparing

In [16]:
data = pd.read_excel('/content/drive/MyDrive/Data/BCG GenAI/data_report_growth.xlsx')
data

,Company,Year,Total Revenue,Net Income,Total Assets,Total Liabilities,Cash Flow,Revenue Growth (%),Net Income Growth (%),Assets Growth (%),Liabilities Growth (%),Cash Flow Growth(%)
0,Microsoft,2023,211915,72361,411976,205753,87582,0.000000,0.000000,0.000000,0.000000,0.000000
1,Microsoft,2022,198270,72738,364840,198298,89035,-6.438902,0.520999,-11.441443,-3.623276,1.659017
2,Microsoft,2021,168088,61271,333779,191791,76740,-15.222676,-15.764800,-8.513595,-3.281425,-13.809176
3,Tesla,2023,96773,14997,106618,43009,13256,0.000000,0.000000,0.000000,0.000000,0.000000
4,Tesla,2022,81462,12556,82338,36440,14724,-15.821562,-16.276589,-22.772890,-15.273547,11.074231
5,Tesla,2021,53823,5519,62131,30548,11497,-33.928703,-56.044919,-24.541524,-16.169045,-21.916599
6,Apple,2023,383285,96995,352583,290437,110543,0.000000,0.000000,0.000000,0.000000,0.000000
7,Apple,2022,394328,99803,352755,302083,122151,2.881146,2.894995,0.048783,4.009820,10.500891
8,Apple,2021,365817,94680,351002,287912,104038,-7.230275,-5.133112,-0.496945,-4.691095,-14.828368


#  Chatbot design and data preparation

5 Question are:
1. What is the total revenue of each company ?
2. How has net income changed from 2022 to 2023 for each company?
3. Year with the highest asset growth for each company?
4. What is the average cash flow of each company over the available years?
5. Which company had the highest net income in 2022?
6. Which company had a larger decrease in total assets from 2021 to 2022?



In [34]:
class ChatBot:
  def __init__(self, data):
    self.data = data
    self.reponse = {
        # 'What is revenue': self.total_revenue(),
        'What is the total revenue of each company ?': self.answer1(),
        'How has net income changed from 2022 to 2023 for each company?': self.answer2(),
        'Year with the highest asset growth for each company?': self.answer3(),
        'What is the average cash flow of each company?': self.answer4(),
        'Which company had the highest net income in 2022?': self.answer5(),
        'Which company had a larger decrease in total assets from 2021 to 2022?': self.answer6()
    }
    self.question = ['What is the total revenue of each company ?', 'How has net income changed from 2022 to 2023 for each company?',
                     'Year with the highest asset growth for each company?', 'What is the average cash flow of each company?',
                     'Which company had the highest net income in 2022?', 'Which company had a larger decrease in total assets from 2021 to 2022?']

  def display_question(self):
    for i, question in enumerate(self.question):
      print(f"{i+1}. {question}")
    print('Type "quite" or 0 when you want to stop!')

  def answer1(self):
    return self.data.groupby('Company')['Total Revenue'].sum().reset_index()
    # return df.to_string(index=False)

  def answer2(self):
    return self.data[self.data['Year'] == 2022][['Company', 'Net Income Growth (%)']]

  def answer3(self):
    return self.data.loc[self.data.groupby('Company')['Assets Growth (%)'].idxmax()][['Company', 'Year', 'Assets Growth (%)']].reset_index(drop=True)

  def answer4(self):
    return self.data.groupby('Company')['Cash Flow'].mean().reset_index()

  def answer5(self):
    idx = self.data[self.data['Year'] == 2022]['Net Income'].idxmax()
    return self.data.loc[idx, ['Company', 'Net Income']]

  def answer6(self):
    df = self.data[self.data['Year'].isin([2021,2022])].pivot(index = 'Company', columns = 'Year', values = 'Total Assets')
    return (df[2021] - df[2022]).idxmax()

  def chat(self):
    print('Hello, please choose your question:')
    while True:
      self.display_question()
      choice = input('Pls enter the number (or "quite" to exit): ')
      if choice.lower() == 'quite' or int(choice) == 0:
        print('Bye!')
        break

      if choice.isdigit() and 1 <= int(choice) <= len(self.question):
        print('Your question is {}'.format(self.question[int(choice)-1]))
        print('The answer is:')
        print(self.reponse[self.question[int(choice)-1]])

      print('\n','Pls choose again!')

# Demonstration and documentation

## Demonstration

In [37]:
chatbot = ChatBot(data)
chatbot.chat()

Hello, please choose your question:
1. What is the total revenue of each company ?
2. How has net income changed from 2022 to 2023 for each company?
3. Year with the highest asset growth for each company?
4. What is the average cash flow of each company?
5. Which company had the highest net income in 2022?
6. Which company had a larger decrease in total assets from 2021 to 2022?
Type "quite" or 0 when you want to stop!
Pls enter the number (or "quite" to exit): 4
Your question is What is the average cash flow of each company?
The answer is:
     Company      Cash Flow
0      Apple  112244.000000
1  Microsoft   84452.333333
2      Tesla   13159.000000

 Pls choose again!
1. What is the total revenue of each company ?
2. How has net income changed from 2022 to 2023 for each company?
3. Year with the highest asset growth for each company?
4. What is the average cash flow of each company?
5. Which company had the highest net income in 2022?
6. Which company had a larger decrease in total a

## Brief documentation
* ***Overview:*** The ChatBot class provides answers to financial queries based on a dataset. The chatbot allows users to ask predefined questions about company revenue, net income, asset growth, and cash flow.

* ***How it works:***
1. Initialization: Accepts financial data and defines questions along with their answers.
2. Question Display: Lists available questions for users to choose from.
3. Answering: Each question has a method that performs calculations and returns relevant data.
4. Interaction: A loop allows users to ask questions and receive answers until they decide to exit.
* ***Limitations:***
1. Only supports predefined questions.
2. Missing expected columns in the data may cause errors.
3. Users must input the number corresponding to their question choice.